# Applied Data Science Capstone - Opening a BBQ Restaurant in Kansas City

## Introduction

One of the things that Kansas City is most well known for is their distinct barbecue style - a variety of different meats that are slow smoked and coated in a thick tomato based sauce.  If you ask a Kansas City local what their favorite barbecue restaurant is, you are likely to hear a variety of answers.  There are many different restaurants located around the metropolitan area, and each one has a slightly different take on the style than the others.  However, as the Kansas City metropolitan area continues to grow, there will be additional opportunities for new BBQ restaurants to start up.  The biggest challenges that a new BBQ restaurant would face is standing out within the Kansas City area, and a good location is one of the best ways to address this challenge.  This report is an attempt to help future business developers decide where in Kansas City a new BBQ venue could be placed successfully. 

The major factors we will use to determine the optimal location of a new BBQ restaurant are the following:
1. How many additional BBQ restaurants are within the region of Kansas City?
2. How many restaurants in total are already located within the region of Kansas City?
3. How many people live in this region of Kansas City?

Our ultimate goal is to find a location that is well populated, but that also isn't completely developed with too many restaurants and especially restaurants that would be a direct competitor with the new venue that is being planned.  

By using data science techniques, we will analyze the neighborhoods of the Kansas City metropolitan area on the criteria given above.  Then, based on these criteria, we will make some recommendations of potential locations that the new BBQ restaurant could be placed into, with the positives and negatives of each potential location considered.

## Data

The Kansas City metropolitan area can be divided into Kansas City and a large collection of suburban areas that surround it.  As we would like to place our new restaurant in a well populated area, according to the criteria given in the introduction, let's limit ourselves to towns in the Kansas City metro area with a population above 20,000 people.  This leaves us with the following collection of towns, with given populations, as provided by the wikipedia page https://en.wikipedia.org/wiki/Kansas_City_metropolitan_area:

<ul>
  <li>Kansas City, Missouri (488,943)</li>
  <li>Kansas City, Kansas (152, 938)</li>
  <li>Overland Park, Kansas (191,278)</li>
  <li>Olathe, Kansas (137,472)</li>
  <li>Independence, Missouri (117,306)</li>
  <li>Lee's Summit, Missouri (97,290)</li>
  <li>Shawnee, Kansas (65,513)</li>
  <li>Blue Springs, Missouri (54,945)</li>
  <li>Lenexa, Kansas (53,553)</li>
  <li>Leavenworth, Kansas (36,210)</li>
  <li>Leawood, Kansas (34,659)</li>
  <li>Liberty, Missouri (31,507)</li>
  <li>Raytown, Missouri (29,211)</li>
  <li>Gladstone, Missouri (27,140)</li>
  <li>Grandview, Missouri (25,159)</li>
  <li>Belton, Missouri (23,480)</li>
  <li>Prarie Village, Kansas (22,368)</li>
  <li>Gardner, Kansas (21,583)</li>
  <li>Raymore, Missouri (21,167)</li>
</ul>

While there are numerous other municipalities within the Kansas City area, a majority of these municipalities are located within a 15 minute drive from one of the larger municipalities listed above.  As a result, we will not consider them as large enough to warrant consideration for the location of a thriving new BBQ business within the Kansas City area.

In order to make a decision about which of the following municipalities would make good locations for a new BBQ restaurant, we will need to collect and use the following information:

1.  The geographic latitude and longitude coordinates for each of the cities listed above.  These centers can be obtained by using the geopy package in Python, or additionally, through the Wikipedia pages for the municipalities, all of which are linked in the reference above.

2.  A geographical boundary for each of the following cities.  As each of the above municipalities have different sizes from one another, it will be important to provide a radius in which we want to search for other restaurants and in particular other BBQ restaurants that our new venue would have to compete against.

3.  By applying the Foursquare API, we will collect a list of all restaurants, as well as those which are designated BBQ restaurants within the boundaries that are defined in the previous point.

After all of this data is collected, we can use it to determine which municipalities are the best fit for our new venue.

In the following cells, we compute the geographic centers of the municipalities that we listed above.

In [122]:
! pip install geopy
from geopy.geocoders import Nominatim

In [176]:
import pandas as pd
import numpy as np
columns = ['Name', 'Latitude', 'Longitude']
namesList = ['Kansas City Missouri', 'Kansas City Kansas', 'Overland Park Kansas', 'Olathe Kansas', 'Independence Missouri', "Lee's Summit Missouri", 'Shawnee Kansas', 'Blue Springs Missouri', 'Lenexa Kansas', 'Leavenworth Kansas', 'Leawood Kansas', 'Liberty Missouri', 'Raytown Missouri', 'Gladstone Missouri', 'Grandview Missouri', 'Belton Missouri', 'Prarie Village, Kansas', 'Gardner Kansas', 'Raymore Missouri']
latitudeList = []
longitudeList = []
KCdf = pd.DataFrame(columns = columns)
KCdf['Name'] = namesList
for name in namesList:
    geolocator = Nominatim(user_agent="KCMunicipalities")
    location = geolocator.geocode(name)
    latitudeList.append(location.latitude)
    longitudeList.append(location.longitude)
KCdf['Latitude'] = latitudeList
KCdf['Longitude'] = longitudeList
KCdf

,Name,Latitude,Longitude
0,Kansas City Missouri,39.100105,-94.578142
1,Kansas City Kansas,39.113456,-94.626497
2,Overland Park Kansas,38.974250,-94.685170
3,Olathe Kansas,38.883886,-94.818870
4,Independence Missouri,39.092479,-94.413792
5,Lee's Summit Missouri,38.910716,-94.382130
6,Shawnee Kansas,39.041672,-94.720238
7,Blue Springs Missouri,39.017316,-94.282265
8,Lenexa Kansas,38.969746,-94.784584
9,Leavenworth Kansas,39.311326,-94.922759


Notice that Prarie Village, Kansas is located very far from the other points above.  The geocode method has located the wrong Prarie Village, and so we adjust the data manually for this row of the table.

In [179]:
KCdf.at[16, 'Latitude'] = 38.989722
KCdf.at[16, 'Longitude'] = -94.636111
KCdf[16:17]

,Name,Latitude,Longitude
16,"Prarie Village, Kansas",38.989722,-94.636111


First, let's visualize these points on a map using the Folium package.

In [ ]:
! pip install folium
import folium

In [180]:
KCMap = folium.Map(location = [39.100105, -94.578142], zoom_start = 9, tiles = 'CartoDB positron')
for row in KCdf.itertuples():
    folium.Marker([row.Latitude, row.Longitude], popup= row.Name).add_to(KCMap)

KCMap

The above map provides the latitude and longitude of the centers of the municipalities that we will use for the centers of possible places to locate the new BBQ restaurant.  However, we need to define the boundaries of these municipalities as well, so that we know how far away from the center to search for other restaurants.  

First, let's treat each of the municipalities of Kansas City as a point.

In [181]:
KCPoints = []
for row in KCdf.itertuples():
    rowPoint = (row.Latitude, row.Longitude)
    KCPoints.append(rowPoint)
KCPoints

[(39.100105, -94.5781416),
 (39.1134562, -94.626497),
 (38.9742502, -94.6851702),
 (38.8838856, -94.81887),
 (39.0924792, -94.4137923),
 (38.9107156, -94.3821295),
 (39.0416718, -94.7202376),
 (39.017316, -94.282265),
 (38.9697458, -94.7845837),
 (39.3113257, -94.922759),
 (38.966673, -94.6169012),
 (39.246479, -94.419079),
 (39.0168425, -94.4630468),
 (39.2214765, -94.5719748),
 (38.8898478, -94.5313816),
 (38.8108097, -94.5313502),
 (38.989722, -94.636111),
 (38.8109254, -94.9272958),
 (38.810131, -94.4676501)]

Now, let's compute the distance between these points, with the caveat that we want to create a circle of at least radius 1.5 miles around each of our municipalities.  Then, let's store the minimum distance between these points that is at least three miles or larger. We'll also need to convert this distance into meters, as that is the measurement that Folium accepts.

In [182]:
from geopy import distance
KCdf['minDistance'] = np.nan
i = 0
for point in KCPoints:
    distanceList = []
    for secondpoint in KCPoints:
        if distance.distance(point, secondpoint).miles > 3:
            distanceList.append(distance.distance(point, secondpoint).miles)
    minDistance = min(distanceList)
    KCdf.at[i, 'minDistance'] = minDistance
    i = i+1
KCdf['minDistanceMeters'] = KCdf['minDistance']*1609.34
KCdf

,Name,Latitude,Longitude,minDistance,minDistanceMeters
0,Kansas City Missouri,39.100105,-94.578142,8.228269,13242.083018
1,Kansas City Kansas,39.113456,-94.626497,7.065800,11371.274754
2,Overland Park Kansas,38.974250,-94.685170,3.713225,5975.841168
3,Olathe Kansas,38.883886,-94.818870,6.204160,9984.603159
4,Independence Missouri,39.092479,-94.413792,5.851633,9417.267530
5,Lee's Summit Missouri,38.910716,-94.382130,8.172844,13152.884795
6,Shawnee Kansas,39.041672,-94.720238,5.019238,8077.661186
7,Blue Springs Missouri,39.017316,-94.282265,8.770909,14115.374769
8,Lenexa Kansas,38.969746,-94.784584,5.362264,8629.705748
9,Leavenworth Kansas,39.311326,-94.922759,19.805560,31873.880719


Now, let's plot circles that will represent the areas for each of the various municipalities that we are looking at.

In [183]:
KCMapRadii = folium.Map(location = [39.100105, -94.578142], zoom_start = 9, tiles = 'CartoDB positron')
for row in KCdf.itertuples():
    folium.Marker([row.Latitude, row.Longitude], popup= row.Name).add_to(KCMapRadii)
    folium.Circle(radius = row.minDistanceMeters/2, location = [row.Latitude, row.Longitude], color = 'Blue', fill = True).add_to(KCMapRadii)

KCMapRadii

First, notice that there is some overlap between Kansas City Kansas and Kansas City Missouri.  This is somewhat unavoidable, as the cities bleed into each other, geographically.  There is also overlap between some of the larger Kansas suburbs:  Overland Park, Prarie Village, and Leawood.  Finally, there is a rather large circle surrounding Leavenworth, Kansas.  This is due to the relatively large distance between it and Kansas City.  In reality, we should not have such a large area representing the municipality.  Let's manually shrink the radius to better fit the city, and see what the results look like.

In [184]:
KCdf.at[9, 'minDistance'] = 5
KCdf.at[9, 'minDistanceMeters'] = 5*1609.34
KCdf[9:10]

,Name,Latitude,Longitude,minDistance,minDistanceMeters
9,Leavenworth Kansas,39.311326,-94.922759,5.0,8046.7


In [185]:
KCMapRadii = folium.Map(location = [39.100105, -94.578142], zoom_start = 9, tiles = 'CartoDB positron')
for row in KCdf.itertuples():
    folium.Marker([row.Latitude, row.Longitude], popup= row.Name).add_to(KCMapRadii)
    folium.Circle(radius = row.minDistanceMeters/2, location = [row.Latitude, row.Longitude], color = 'Blue', fill = True).add_to(KCMapRadii)

KCMapRadii

We've now defined a reasonable radius to search for restaurants within for each of our municipalities.  Let's now use the Foursquare API to determine how many different restaurants there are within each of the circles that we've defined.

In [224]:
import requests
CLIENT_ID = 'CSFSY4SGBL0IYHGPZXSDCY3G2YBBI5DT54LU1UHR0WZX3JUT' # your Foursquare ID
CLIENT_SECRET = 'R3RV53OJXUN2XMO2S52B4R1A4DX0F152BZQBSPHWN1Y05V3G' # your Foursquare Secret
VERSION = '20180604'
print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: CSFSY4SGBL0IYHGPZXSDCY3G2YBBI5DT54LU1UHR0WZX3JUT
CLIENT_SECRET:R3RV53OJXUN2XMO2S52B4R1A4DX0F152BZQBSPHWN1Y05V3G


The cell below defines a call to the Foursquare API that finds all of the restaurants within a specified radius of a point that are tagged as a BBQ Joint.  It then takes those restaurants and places them into a table. 

In [225]:

def getNearbyBBQJoints(lat, lng, rad):
    
    venues_list=[]
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, '4bf58dd8d48988d1df931735', rad, 1000)
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    venues_list.append([(
        v['venue']['name'], 
        v['venue']['location']['lat'], 
        v['venue']['location']['lng'],  
        v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [ 
                  'Venue', 
                  'Venue_Latitude', 
                  'Venue_Longitude', 
                  'Venue_Category']
    nearby_venues = nearby_venues[nearby_venues.Venue_Category == 'BBQ Joint']
    return(nearby_venues)

In the cell below, we'll find all of the restaurants within the different municipalities that we considered at the start of the project, and place them on a map.

In [226]:
for i in range(18):
    print(getNearbyBBQJoints(KCdf.at[i, 'Latitude'], KCdf.at[i, 'Longitude'], KCdf.at[i, 'minDistanceMeters']/2))
    

                             Venue  Venue_Latitude  Venue_Longitude  \
0   Fiorella's Jack Stack Barbecue       39.087455       -94.585568   
1         Arthur Bryant's Barbeque       39.091446       -94.556285   
2            County Road Ice House       39.097412       -94.582144   
3                    Gates Bar-B-Q       39.098435       -94.555813   
4                       Slap's BBQ       39.102547       -94.624463   
5                              Q39       39.057394       -94.598225   
6                    Gates Bar-B-Q       39.069013       -94.585127   
7                         Char Bar       39.053999       -94.592560   
8              Hawg Jaw Que & Brew       39.126202       -94.577500   
9                Plowboys Barbeque       39.100123       -94.583051   
10                    Rosedale BBQ       39.065897       -94.616166   
11      Three Little Pigs Barbeque       39.082738       -94.581730   
12                   Gates Bar-B-Q       39.116787       -94.636172   
13    

                      Venue  Venue_Latitude  Venue_Longitude Venue_Category
0   R.J.'s Bob-Be-Que Shack       39.022798       -94.658150      BBQ Joint
1                  C. Frogs       39.002000       -94.631150      BBQ Joint
2              Johnny's BBQ       39.020794       -94.665211      BBQ Joint
8         KC Smokehouse BBQ       38.974862       -94.594961      BBQ Joint
10       Wyandot Barbeque 2       38.992845       -94.669574      BBQ Joint
        Venue  Venue_Latitude  Venue_Longitude Venue_Category
2  Billy Sims       38.812739       -94.906339      BBQ Joint


In [233]:
KCMapBBQ = folium.Map(location = [39.100105, -94.578142], zoom_start = 9, tiles = 'CartoDB positron')
for row in KCdf.itertuples():
    folium.Marker([row.Latitude, row.Longitude], popup= row.Name).add_to(KCMapBBQ)
    folium.Circle(radius = row.minDistanceMeters/2, location = [row.Latitude, row.Longitude], color = 'Blue', fill = True).add_to(KCMapBBQ)
for i in range(18):
    df = getNearbyBBQJoints(KCdf.at[i, 'Latitude'], KCdf.at[i, 'Longitude'], KCdf.at[i, 'minDistanceMeters']/2)
    for row in df.itertuples():
        folium.Marker([row.Venue_Latitude, row.Venue_Longitude], popup= row.Venue, icon=folium.Icon(color='green')).add_to(KCMapBBQ)
KCMapBBQ

In the cells below, we'll compute the restaurants that are located within each of the municipalities we have considered.

In [220]:
def getNearbyRestaurants(lat, lng, rad):
    
    venues_list=[]
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, '4d4b7105d754a06374d81259', rad, 1000)
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    venues_list.append([(
        v['venue']['name'], 
        v['venue']['location']['lat'], 
        v['venue']['location']['lng'],  
        v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [ 
                  'Venue', 
                  'Venue_Latitude', 
                  'Venue_Longitude', 
                  'Venue_Category']
    return(nearby_venues)

In [221]:
for i in range(18):
    print(getNearbyRestaurants(KCdf.at[i, 'Latitude'], KCdf.at[i, 'Longitude'], KCdf.at[i, 'minDistanceMeters']/2))

                                   Venue  Venue_Latitude  Venue_Longitude  \
0                  Bristol Seafood Grill       39.097064       -94.582879   
1                                  Zaina       39.100654       -94.580862   
2             Milwaukee Delicatessen Co.       39.103463       -94.584324   
3                             Tavernonna       39.100081       -94.584549   
4                             Yard House       39.098491       -94.583226   
5                               Grinders       39.091405       -94.578175   
6                         Blue Nile Café       39.108569       -94.582190   
7     Corvino Supper Club & Tasting Room       39.090617       -94.582752   
8            Happy Gillis Cafe & Hangout       39.108916       -94.570382   
9                           il Lazzarone       39.108572       -94.584449   
10                  Garozzo's Ristorante       39.109492       -94.571955   
11                               Beignet       39.109559       -94.583027   

                                                Venue  Venue_Latitude  \
0                               Thai Place Restaurant       38.970816   
1                                        India Palace       38.971607   
2                                        Pizza Shoppe       38.971951   
3            J. Gilbert's Wood-Fired Steaks & Seafood       38.967260   
4                                          Dragon Inn       38.984069   
5                                         Mi Ranchito       38.984278   
6                       Mr. Gyros Greek Food & Pastry       38.978801   
7                                    Lenny's Sub Shop       38.970807   
8                         The Snack Shack on Santa Fe       38.983103   
9                                            ABC Cafe       38.971013   
10                                 Clock Tower Bakery       38.985322   
11                                       Maui Express       38.973002   
12                               Papa Keno's Pizzer

                           Venue  Venue_Latitude  Venue_Longitude  \
0                    Cafe Verona       39.092208       -94.417490   
1                      Ophelia's       39.092804       -94.415698   
2            Courthouse Exchange       39.092076       -94.416382   
3                   Square Pizza       39.092940       -94.417582   
4             A Little BBQ Joint       39.101690       -94.428947   
5                      Rheinland       39.093184       -94.415722   
6               Mugs Up Drive In       39.079195       -94.406644   
7                 Mandarin House       39.079720       -94.436977   
8                 HiBoy Drive-in       39.102342       -94.427107   
9                       Vivilore       39.087602       -94.449800   
10          Bill & Ann's Mugs Up       39.079200       -94.406649   
11             Back Yard Burgers       39.078553       -94.388742   
12                Minsky's Pizza       39.073406       -94.378299   
13          Dave's Bakery & Deli  

                                     Venue  Venue_Latitude  Venue_Longitude  \
0      Old Shawnee Pizza & Italian Kitchen       39.019134       -94.714260   
1                        Big Bam's Burgers       39.020937       -94.714801   
2                           Firehouse Subs       39.013765       -94.721469   
3                               Sushi Mido       39.019573       -94.714900   
4                              Oishi Sushi       39.015615       -94.727416   
5                            Blind Box BBQ       39.015752       -94.739474   
6                     Pine & Bamboo Garden       39.014462       -94.712501   
7                            El Pulgarcito       39.021188       -94.693973   
8                       Grandstand Burgers       39.039507       -94.686353   
9                          The Big Biscuit       39.015242       -94.728068   
10                             First Watch       39.015069       -94.716273   
11                              Dos Reales       39.

                                     Venue  Venue_Latitude  Venue_Longitude  \
0                                 LifeCafe       38.964386       -94.781316   
1                   SPIN! Neapolitan Pizza       38.957162       -94.780545   
2                   Ignite Wood Fire Grill       38.969597       -94.777840   
3   Freddy's Frozen Custard & Steakburgers       38.971698       -94.774410   
4                        Grand Street Cafe       38.968639       -94.778909   
5                                Lucky Wok       38.970270       -94.761952   
6                             Panera Bread       38.970004       -94.762692   
7                           Minsky's Pizza       38.998257       -94.780601   
8                           Paulo and Bill       38.998895       -94.777428   
9                            Jose Pepper's       38.998450       -94.778165   
10                              Brewbakers       38.956985       -94.780649   
11                          Hereford House       39.

                                   Venue  Venue_Latitude  Venue_Longitude  \
0                            Chick-fil-A       38.973380       -94.607780   
1                              Five Guys       38.970423       -94.607259   
2                           Charleston’s       38.967914       -94.607025   
3                         Firehouse Subs       38.970856       -94.607398   
4                           Panera Bread       38.977914       -94.631124   
5                          Dewey's Pizza       38.956498       -94.627741   
6                            Red Snapper       38.975384       -94.607249   
7             O'Neill's Restaurant & Bar       38.958105       -94.629971   
8                        Smitty's Garage       38.968223       -94.606596   
9                     BRGR Kitchen + Bar       38.979629       -94.632259   
10               Great Harvest Bread Co.       38.979593       -94.631513   
11                SPIN! Neapolitan Pizza       38.980262       -94.630475   

                             Venue  Venue_Latitude  Venue_Longitude  \
0        Goodcents Deli Fresh Subs       39.009376       -94.461561   
1     El Maguey Mexican Restaurant       39.003263       -94.463989   
2                      Las Chili's       39.010293       -94.467226   
3           Chipotle Mexican Grill       38.994398       -94.474550   
4                   Firehouse Subs       39.046640       -94.446022   
5                       Mama China       39.002572       -94.463310   
6   Salvatore's Italian Ristorante       39.038579       -94.429263   
7                  Big Boy Burgers       39.042031       -94.433590   
8                     Jimmy John's       38.994305       -94.474610   
9      Ridgewood Donuts and Bakery       39.043076       -94.437746   
10           Fun House Pizza & Pub       39.037470       -94.426209   
11                Peachtree Buffet       39.031134       -94.504341   
12                   Gates Bar-B-Q       39.053308       -94.454891   
13    

                           Venue  Venue_Latitude  Venue_Longitude  \
0               Providence Pizza       38.889100       -94.523621   
1   Campestre Mexican Restaraunt       38.889612       -94.536552   
2                    Captain D's       38.888374       -94.523269   
3               McAlister's Deli       38.901354       -94.525357   
4         Chipotle Mexican Grill       38.902037       -94.525274   
5                         Subway       38.902779       -94.530848   
6               Grandview Buffet       38.900491       -94.532297   
7              Papa John's Pizza       38.906342       -94.527022   
8                      Fish City       38.914128       -94.514596   
9           Little Caesars Pizza       38.888280       -94.521503   
10                  Waffle House       38.889249       -94.526928   
11                SONIC Drive In       38.858787       -94.522668   
12                     Pizza Hut       38.873050       -94.527578   
13                     Pizza Hut  

                                       Venue  Venue_Latitude  Venue_Longitude  \
0                         BRGR Kitchen + Bar       38.979629       -94.632259   
1                               Panera Bread       38.977914       -94.631124   
2                              Cafe Provence       39.002451       -94.631419   
3                                     story.       39.002097       -94.631349   
4                    Great Harvest Bread Co.       38.979593       -94.631513   
5                     SPIN! Neapolitan Pizza       38.980262       -94.630475   
6                               Dolce Bakery       39.002498       -94.631343   
7                      Tavern in the Village       39.003475       -94.630660   
8                                Urban Table       38.979665       -94.631326   
9                                Caffetteria       39.001698       -94.630971   
10                 Jun's Japanese Restaurant       38.989226       -94.609214   
11                          

Using the restaurant data that we have found above, along with the specific restaurant data for BBQ Joints and the populations of the municipalities surrounding Kansas City, we will analyze potential places for the ideal new BBQ restaurant.